In [2]:
!pip3 install missingno
!pip install pandas==1.3.5

# Load data from s3

In [1]:
import io
import boto3
import configparser

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 200)

def read_csv_from_s3(section_name, bucket_name, file_path, ):
    
    parser = configparser.ConfigParser()
    parser.read('./config.ini')
    access_key = parser.get(section_name, "access_key")
    secret_key = parser.get(section_name, "secret_key")
    
    s3 = boto3.client('s3',
                      aws_access_key_id=access_key, 
                      aws_secret_access_key=secret_key)
    print("Get object")
    obj = s3.get_object(Bucket=bucket_name, Key=file_path)

    print("Read csv")
    df = pd.read_csv( io.BytesIO( obj["Body"].read() ) )
    
    print("Completed")
    print("Data size:", df.shape)

    return df

In [2]:
def write_csv_to_s3(section_name, bucket_name, file_path, data):
    
    parser = configparser.ConfigParser()
    parser.read('./config.ini')
    access_key = parser.get(section_name, "access_key")
    secret_key = parser.get(section_name, "secret_key")
    
    s3 = boto3.client('s3',
                      aws_access_key_id=access_key, 
                      aws_secret_access_key=secret_key)

    print("Put object")
    print("Data Shape: ", data.shape)
    s3.put_object(
        Body=data.to_csv(index=False).encode(),
        Bucket=bucket_name, Key=file_path)

    print("Completed")

In [3]:
section_name = "aws_boto_credentials"
bucket_name = "ev-depreciation-model"
file_path = "modeling/220701_used_car_v2.csv"

df = read_csv_from_s3(section_name, bucket_name, file_path)

Get object
Read csv
Completed
Data size: (374526, 20)


In [4]:
df = df.dropna(subset=["manufacturer", "model"])
df.shape

(356515, 20)

# Price of New Car

## EV manufacturer & model

In [5]:
ev = df.fuel.isin(["electric", "hybrid"])
df_ev = df[ev][["manufacturer", "model", "fuel", "drive", "type"]].drop_duplicates(keep="last").sort_values(["manufacturer", "model"])

file_path = "analysis/220705_ev_model_unique.csv"
write_csv_to_s3(section_name, bucket_name, file_path, df_ev)

Put object
Data Shape:  (1187, 5)
Completed


In [6]:
mf5 = list(df[ev].manufacturer.value_counts().head(5).index)
ev_mf5 = df.manufacturer.isin(mf5)
df_ev_mf5 = df[ev & ev_mf5]
df_ev_mf5.sample(10)

,price,year,manufacturer,model,odometer,condition,title_status,cylinders,fuel,transmission,drive,size,type,paint_color,region,state,VIN,checked_VIN,posting_year,posting_day
310446,38990,2020.0,tesla,model 3 standard range,9665.0,good,clean,NaN,electric,other,NaN,NaN,sedan,white,knoxville,tn,5YJ3E1EA7LF504671,1,2021.0,124.0
26241,644,2018.0,tesla,model s 75d - autopilot,19341.0,NaN,clean,NaN,electric,automatic,4wd,NaN,hatchback,NaN,inland empire,ca,5YJSA1E22JF246704,1,2021.0,123.0
5331,10495,2016.0,ford,c-max hybrid,71835.0,NaN,clean,4 cylinders,hybrid,automatic,fwd,NaN,wagon,custom,anchorage / mat-su,ak,1FADP5AU2GL100322,1,2021.0,118.0
345633,5997,2007.0,toyota,prius,171720.0,excellent,clean,4 cylinders,hybrid,automatic,fwd,NaN,hatchback,NaN,norfolk / hampton roads,va,JTDKB20U773269350,1,2021.0,118.0
339421,31200,2018.0,tesla,model 3,26105.0,NaN,NaN,NaN,electric,automatic,rwd,full-size,sedan,NaN,salt lake city,ut,5YJ3E1EB7JF113221,1,2021.0,105.0
348909,10500,2013.0,toyota,prius,69989.0,NaN,clean,4 cylinders,hybrid,automatic,rwd,NaN,NaN,silver,richmond,va,JTDKN3DU8D5608155,1,2021.0,95.0
22175,15998,2017.0,ford,c-max energi,28200.0,excellent,clean,NaN,hybrid,automatic,NaN,NaN,NaN,NaN,chico,ca,1FADP5EU5HL118230,1,2021.0,96.0
331522,20995,2018.0,ford,c-max hybrid,20919.0,NaN,clean,NaN,hybrid,automatic,fwd,NaN,hatchback,NaN,laredo,tx,1FADP5AU4JL101981,1,2021.0,120.0
39033,29990,2013.0,tesla,model s sedan 4d,59613.0,good,clean,NaN,electric,other,rwd,NaN,sedan,blue,redding,ca,5YJSA1CG5DFP16162,1,2021.0,124.0
277234,563,2013.0,tesla,model s base,34162.0,NaN,clean,NaN,electric,automatic,rwd,NaN,sedan,NaN,corvallis/albany,or,5YJSA1CG1DFP03876,1,2021.0,96.0


In [8]:
file_path = "analysis/220705_ev5_model_unique.csv"

df_sample = df_ev_mf5[["manufacturer", "model"]].drop_duplicates(keep="last").sort_values(["manufacturer", "model"])
write_csv_to_s3(section_name, bucket_name, file_path, df_sample)

Put object
Data Shape:  (401, 2)
Completed


## 1. toyota

In [5]:
toyota = df_ev_mf5.manufacturer==mf5[0]

In [7]:
df_ev_mf5[toyota].model.value_counts().sort_index()

avalon                           4
avalon hybrid                   11
avalon hybrid limited           29
avalon hybrid xle                2
avalon hybrid xle plus           8
avalon hybrid xle premium        2
avalon hybrid xle touring        1
avalon hybrid xse sedan          4
avalon limited hybrid            1
avalon xle                       1
avalon xle sedan 4d              1
camry                           41
camry hybrid                    63
camry hybrid base                1
camry hybrid le                  7
camry hybrid le sedan            1
camry hybrid le sedan 4d         7
camry hybrid se                 14
camry hybrid sedan               4
camry hybrid sedan 4d            1
camry hybrid xle                10
camry hybrid xle sedan 4d        6
camry hybrid | gas saver,        1
camry le                         4
camry le sedan 4d               10
corolla                          4
corolla hybrid                   1
electric rav4 prime xse awd      5
highlander          

### 1.1. avalon

* 승용 
  * avalon - 6b
  * avalon - BK36B
  * avalon - GSX40L
  * avalon - LIMITED (avalon limited)
  * avalon - MCX10L
  * avalon - XL
  * avalon hybrid - AXXH50L

* 공식 사이트 (avalon msrp)
    * xle (36825)
    * xle hybrid (37850)
    * xse hybrid nightshade (41200)
    * touring (43525)
    * limited (43125)
    * limited hybrid (44150)

In [32]:
avalon = df_ev_mf5.model.str.contains("avalon")
df_ev_mf5[toyota & avalon].model.value_counts()

avalon hybrid limited        29
avalon hybrid                11
avalon hybrid xle plus        8
avalon                        4
avalon hybrid xse sedan       4
avalon hybrid xle             2
avalon hybrid xle premium     2
avalon limited hybrid         1
avalon hybrid xle touring     1
avalon xle                    1
avalon xle sedan 4d           1
Name: model, dtype: int64